<a href="https://colab.research.google.com/github/RushiBShinde/ThinkForge-IE-643-project/blob/main/Quantifying_Relevance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arxiv beautifulsoup4 requests sentence-transformers scikit-learn nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ae8a82093b049621fa288321494f5ec471f2962b38d14a4f8178aec69575363b
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
!pip install PyMuPDF requests sentence-transformers scikit-learn nltk rank_bm25

In [ ]:
!pip install PyMuPDF requests sentence-transformers scikit-learn nltk

In [ ]:
import requests
import fitz  # PyMuPDF
import io
import nltk
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import numpy as np
import re
import time

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Fetching the text from papers

In [ ]:
def get_paper_text(pdf_url):
    """
    Fetches a PDF from a URL and extracts its text content.
    """
    try:
        # 1. Download the PDF content from the URL
        response = requests.get(pdf_url)
        response.raise_for_status()  # Check for any download errors

        # 2. Open the downloaded content in memory
        pdf_document = fitz.open(stream=io.BytesIO(response.content), filetype="pdf")

        # 3. Extract text from each page and combine it
        full_text = ""
        for page in pdf_document:
            full_text += page.get_text()

        pdf_document.close()
        return full_text

    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF from {pdf_url}: {e}")
        return ""
    except Exception as e:
        print(f"Error processing PDF from {pdf_url}: {e}")
        return ""

# List of on-topic and off-topic paper URLs (now linking directly to the PDFs)
paper_links = {
    "Attention Is All You Need": "https://arxiv.org/pdf/1706.03762.pdf",
    "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding": "https://arxiv.org/pdf/1810.04805.pdf",
    "Reformer: The Efficient Transformer": "https://arxiv.org/pdf/2001.04451.pdf",
    "Programmable time-frequency mode-sorting of single photons with a multi-output quantum pulse gate": "https://arxiv.org/pdf/2410.03606.pdf",
    "Testing the space-time geometry around black hole candidates with the available radio and X-ray data": "https://arxiv.org/pdf/1301.0361.pdf",
    "Optimal Design of Power Electronic Transformer based on Hybrid MMC under Boost-AC operation":"https://arxiv.org/pdf/2405.17291"
}


papers = {}
for title, url in paper_links.items():
    print(f"Fetching and processing: {title}")
    text = get_paper_text(url)
    if text:
        papers[title] = text
        print(f"Successfully processed '{title}'")
    else:
        print(f"Could not process: {title}")

# Let's check a snippet from one of the fetched papers
print("\n--- Snippet from 'Attention Is All You Need' ---")
if "Attention Is All You Need" in papers:
    # Clean up the text a bit for preview by removing excessive newlines
    clean_snippet = re.sub(r'\n\s*\n', '\n', papers["Attention Is All You Need"][:1000])
    print(clean_snippet)

Fetching and processing: Attention Is All You Need
Successfully processed 'Attention Is All You Need'
Fetching and processing: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Successfully processed 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding'
Fetching and processing: Reformer: The Efficient Transformer
Successfully processed 'Reformer: The Efficient Transformer'
Fetching and processing: Programmable time-frequency mode-sorting of single photons with a multi-output quantum pulse gate
Successfully processed 'Programmable time-frequency mode-sorting of single photons with a multi-output quantum pulse gate'
Fetching and processing: Testing the space-time geometry around black hole candidates with the available radio and X-ray data
Successfully processed 'Testing the space-time geometry around black hole candidates with the available radio and X-ray data'
Fetching and processing: Optimal Design of Power Electronic Transf

Implementing the Ranking Metrics

In [ ]:
# --- Metric 1: On-Topic Rate (from before) ---
def calculate_on_topic_rate(query, doc_text):
    query_keywords = set(query.lower().split())
    sentences = nltk.sent_tokenize(doc_text)
    if not sentences: return 0.0
    on_topic_count = sum(1 for s in sentences if any(k in s.lower() for k in query_keywords))
    return on_topic_count / len(sentences)

# --- Metric 2: TF-IDF Similarity (from before) ---
def calculate_tfidf_similarity(query, documents):
    docs_list = list(documents.values())
    vectorizer = TfidfVectorizer(stop_words='english')
    doc_vectors = vectorizer.fit_transform(docs_list)
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, doc_vectors)
    return {list(documents.keys())[i]: similarities[0][i] for i in range(len(documents))}

# --- Metric 3: Sentence Embedding Similarity (Bi-Encoder, from before) ---
bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')
def calculate_bi_encoder_similarity(query, documents):
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    doc_embeddings = bi_encoder.encode(list(documents.values()), convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, doc_embeddings)
    return {list(documents.keys())[i]: similarities[0][i].item() for i in range(len(documents))}

# --- Metric 4: BM25 (New) ---
def calculate_bm25_scores(query, documents):
    tokenized_corpus = [doc.split(" ") for doc in documents.values()]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    return {list(documents.keys())[i]: doc_scores[i] for i in range(len(documents))}

# --- Metric 5: Cross-Encoder (New) ---
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
def calculate_cross_encoder_scores(query, documents):
    model_input = [[query, doc_text] for doc_text in documents.values()]
    scores = cross_encoder.predict(model_input)
    return {list(documents.keys())[i]: scores[i] for i in range(len(documents))}

# Ranking and Comparing the *papers*

In [ ]:
# query = "output voltage of transformers "
query = "what are the input embeddings for transformers"

print("--- Calculating scores for all metrics... ---")
start_time = time.time()

# Calculate scores for each metric
otr_scores = {title: calculate_on_topic_rate(query, text) for title, text in papers.items()}
tfidf_scores = calculate_tfidf_similarity(query, papers)
bi_encoder_scores = calculate_bi_encoder_similarity(query, papers)
bm25_scores = calculate_bm25_scores(query, papers)
cross_encoder_scores = calculate_cross_encoder_scores(query, papers)

end_time = time.time()
print(f"--- All calculations finished in {end_time - start_time:.2f} seconds ---\n")

# Rank by each metric
ranked_otr = sorted(otr_scores.items(), key=lambda item: item[1], reverse=True)
ranked_tfidf = sorted(tfidf_scores.items(), key=lambda item: item[1], reverse=True)
ranked_bi_encoder = sorted(bi_encoder_scores.items(), key=lambda item: item[1], reverse=True)
ranked_bm25 = sorted(bm25_scores.items(), key=lambda item: item[1], reverse=True)
ranked_cross_encoder = sorted(cross_encoder_scores.items(), key=lambda item: item[1], reverse=True)

# --- Display the results ---
print("="*60)
print("---               FINAL RANKING COMPARISON               ---")
print("="*60 + "\n")
print(f"--- Query ---\n'{query}'\n")

print("--- Ranking by On-Topic Rate ---")
for title, score in ranked_otr: print(f"{score:.4f} - {title}")

print("\n--- Ranking by TF-IDF Cosine Similarity ---")
for title, score in ranked_tfidf: print(f"{score:.4f} - {title}")

print("\n--- Ranking by BM25 ---")
for title, score in ranked_bm25: print(f"{score:.4f} - {title}")

print("\n--- Ranking by Sentence Embedding (Bi-Encoder) ---")
for title, score in ranked_bi_encoder: print(f"{score:.4f} - {title}")

print("\n--- Ranking by Cross-Encoder () ---")
for title, score in ranked_cross_encoder: print(f"{score:.4f} - {title}")

--- Calculating scores for all metrics... ---
--- All calculations finished in 3.69 seconds ---

---               FINAL RANKING COMPARISON               ---

--- Query ---
'what are the input embeddings for transformers'

--- Ranking by On-Topic Rate ---
0.6654 - Testing the space-time geometry around black hole candidates with the available radio and X-ray data
0.6341 - Attention Is All You Need
0.6299 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
0.6298 - Reformer: The Efficient Transformer
0.5337 - Programmable time-frequency mode-sorting of single photons with a multi-output quantum pulse gate
0.4863 - Optimal Design of Power Electronic Transformer based on Hybrid MMC under Boost-AC operation

--- Ranking by TF-IDF Cosine Similarity ---
0.0736 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
0.0706 - Attention Is All You Need
0.0480 - Programmable time-frequency mode-sorting of single photons with a multi-outpu

==================================================================




==================================================================

# BertScore

In [1]:
!pip install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [2]:
from bert_score import score

cands = ["My name is Rushikesh."]
refs = ["people know me by Rushikesh."]

P, R, F1 = score(cands, refs, lang="en", verbose=True)
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.61 seconds, 1.64 sentences/sec
Precision: 0.9203
Recall:    0.9166
F1:        0.9184


In [5]:
from bert_score import score

cands = ["I like coding in Python.."]
refs = ["Python is my favorite programming language."]

P, R, F1 = score(cands, refs, lang="en", verbose=True)
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.70 seconds, 1.44 sentences/sec
Precision: 0.8821
Recall:    0.8987
F1:        0.8903


In [6]:
from bert_score import score

cands = ["I wake up early in the morning."]
refs = ["My friend likes to swim in cold water."]

P, R, F1 = score(cands, refs, lang="en", verbose=True)
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.81 seconds, 1.24 sentences/sec
Precision: 0.9031
Recall:    0.8956
F1:        0.8994


In [3]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden_dim]

# Candidate and reference
cand = "I like coding in Python."
ref  = "Python is my favorite programming language."

emb_cand = get_embeddings(cand)
emb_ref  = get_embeddings(ref)

# Compute token-to-token cosine similarity
sim_matrix = cosine_similarity(emb_cand, emb_ref)

# Precision = for each cand token, best match in ref
precision = np.mean(np.max(sim_matrix, axis=1))

# Recall = for each ref token, best match in cand
recall = np.mean(np.max(sim_matrix, axis=0))

# F1 score
f1 = 2 * precision * recall / (precision + recall)

print(sim_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1:        {f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

[[ 0.9558883   0.6201248   0.1262367   0.19128278  0.13052134  0.02260007
   0.05227136 -0.06400371 -0.09859119]
 [ 0.539181    0.537478    0.31022847  0.59118056  0.3160471   0.08166911
   0.12076893 -0.02774332 -0.04737328]
 [ 0.07991838  0.00608927  0.46076274  0.39738104  0.50903696  0.20066804
   0.27933526  0.0802674   0.08752699]
 [ 0.19219188  0.349705    0.37722385  0.3498713   0.37090057  0.5697473
   0.47621676  0.07159437  0.0573954 ]
 [ 0.09614316  0.16584218  0.49421635  0.35128266  0.3789055   0.45594049
   0.5350144   0.14816868  0.16386242]
 [ 0.16714859  0.5542229   0.34560835  0.29640478  0.31367898  0.62326694
   0.6435401   0.12921706  0.13252863]
 [-0.03371047  0.02619676  0.15793036  0.12387324  0.09389699  0.134309
   0.13349028  0.944809    0.8814361 ]
 [-0.10785044 -0.04806409  0.14593989  0.09851871  0.07079547  0.14156821
   0.13408259  0.9605805   0.95561177]]
Precision: 0.7137
Recall:    0.7059
F1:        0.7098


In [4]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden_dim]

# Candidate and reference
cand = "My name is Rushikesh."
ref  = "People know me by Rushikesh."

emb_cand = get_embeddings(cand)
emb_ref  = get_embeddings(ref)

# Compute token-to-token cosine similarity
sim_matrix = cosine_similarity(emb_cand, emb_ref)

# Precision = for each cand token, best match in ref
precision = np.mean(np.max(sim_matrix, axis=1))

# Recall = for each ref token, best match in cand
recall = np.mean(np.max(sim_matrix, axis=0))

# F1 score
f1 = 2 * precision * recall / (precision + recall)

print(sim_matrix)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1:        {f1:.4f}")


[[ 0.9343264   0.62380457  0.12391266  0.19797522  0.11650202  0.12669946
   0.07280914  0.12669384  0.09205876 -0.10238526]
 [ 0.642303    0.6391895   0.2912181   0.49375233  0.25682336  0.19206269
   0.13020676  0.1920404   0.2165716  -0.03747476]
 [ 0.12531266  0.09586737  0.4365928   0.4144434   0.52720857  0.24145174
   0.26495895  0.24743162  0.20752923  0.10795635]
 [ 0.14581579  0.14875865  0.49294287  0.49820453  0.5341596   0.28486896
   0.299863    0.3181371   0.26269528  0.11178941]
 [ 0.23833308  0.31470418  0.18749103  0.3015141   0.1725321   0.85653067
   0.38349074  0.43537682  0.14261091  0.03469526]
 [ 0.04381658  0.07405748  0.2563045   0.29583645  0.2611571   0.4943097
   0.9071715   0.501325    0.16272533  0.1095289 ]
 [ 0.10715036  0.16251883  0.3350424   0.34263706  0.28679827  0.5007713
   0.51213455  0.8955194   0.30896053  0.06751893]
 [ 0.04651468  0.17933142  0.2878078   0.25103003  0.24354373  0.18483621
   0.20267029  0.29105723  0.91339135  0.18776017]
 [

In [ ]:
!pip install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
